In [1]:
import pandas as pd

f_train = pd.read_csv('맛집리뷰text2.txt', sep='\t')
train_pair = [(row[0], row[1]) for _, row in f_train.iterrows() if type(row[0]) == str]

train_data  = [pair[0] for pair in train_pair]
train_label = [pair[1] for pair in train_pair]

In [2]:
word_tk = pd.read_csv('SentiWord_Dict.txt', sep='\t')
word_tk

(-;  1
0      (;_;) -1
1       (^^)  1
2      (^-^)  1
3       (^^*  1
4      (^_^)  1
...      ... ..
14849     갈등 -1
14850     의혹 -1
14851  내팽개치다 -2
14852     횡령 -2
14853    불안증 -2

[14854 rows x 2 columns]

In [3]:
p=0
for i in train_label:
    if i == 1:
        p += 1
print(p)
        

388


In [4]:
# 추출된 문장 및 라벨 데이터 확인

for data, label in zip(train_data[:], train_label[:]):
    print(f'문장: {data}\n라벨: {label}\n')

문장: 가성비 최고에 맛까지 좋은 노포 식당입니다..
라벨: 1

문장: 보통의 닭곰탕집들처럼 노계나 폐닭을 사용하지 않고 일반 육계를 사용해서 잡내없고 육질이 참 부드럽습니다..
라벨: 1

문장: 멸치육수베이스의 칼국수도 딱 멸치육수의 교과서같은 맛입니다..
라벨: 1

문장: 익숙한 맛이고 대중적으로 호불호가 갈리지 않을맛..그러면서도 자꾸 생각나게 만드는 맛있는 육수맛입니다..면발도 노포스타일이라 명동교자처럼 아주 부드럽고 매끄러운 스타일입니다..겉절이김치도 보기와 달리 맵거나 짜지않고 살짝 달달하고 맛있습니다..가성비와 맛에 더해 옛추억까지 소환되는 노포 찐맛집..무조건 강추죠..이런 식당은
라벨: 1

문장: 가성비가 좋아요음식이 맛있어요친절해요
라벨: 1

문장: 웨이팅도 너무 길고 음식도 그닥..
라벨: 0

문장: 낮에 혼자 가서 백반 하나 먹었어요. 가성비최고. 혼자손님들 저 말고도 꽤 오더라고요. 엄청 소주 시키고 싶었는데 다음 일정이 있어서 참았… 그리고 파 담긴 접시 주시면서 소스 담아드세요라고 하셨는데 저는 그 접시가 소스접시인 줄 알고;; 냄비안에 넣어버림. 속으로 소스를 넣었는데 왜 그냥 파만 넣은 느낌이 나지? 그러다가 다 먹을 때쯤 옆에 빨간통이 소스라는 사실을 깨달음. 그 소스를 파 담긴 접시에 넣고 섞어먹으라는 말씀이셨음.ㅎㅎㅎㅎㅎㅎ 국물이 뜨끈하면 더 좋겠다 싶었지만 바쁘다바뻐 현대사회에서 후다닥 먹고가기 좋은 메뉴였음.
라벨: 1

문장: 가성비가 좋아요혼밥하기 좋아요
라벨: 1

문장: 진심 맛있어요. 기대없이 갔는데 닭도 야들야들하고 간도 딱이에요.진심 김치도 달달매콤하니 간아주 제 대로입니다. 국물에 밥 말고 닭가슴살 찢어서
라벨: 1

문장: 친절해요
라벨: 1

문장: 친구랑 두명이서 갔는데 백숙백반 2인(인당 8,000원) , 칼국수 1인분(6,000원) 시켜서 먹으니 맛있고 배불 리 잘 먹었습니다 !!
라벨: 1

문장: 음식이 맛있어요
라벨: 1

문장: 백숙백반이 8,000원인데 이렇게 많은 양이 나왔어요

In [5]:
# 프로젝트 전반에 사용될 변수 사전 정의

params = {
    'batch_size': 64,
    'num_epoch': 15,
    'lr': 0.003,
    'dropout': 0.5,
    'min_frequency': 3,
    'max_len': 16,
    
    'vocab_size': 20000,
    'embed_dim': 100,
    'hidden_dim': 256,
    'filter_sizes': [2, 3, 4],
    'num_filters': 100,
    'output_dim': 1,
}

In [6]:
# 추출한 문장 데이터 텍스트 파일로 저장

with open('train_tokenizer.txt', 'w', encoding='utf-8') as f:
    for line in train_data:
        print(line, file=f)

In [7]:
from tokenizers import BertWordPieceTokenizer

# BertWordPieceTokenizer 초기화

tokenizer = BertWordPieceTokenizer()

# 맛집 데이터 텍스트 파일 활용해 토크나이저 훈련

trainer = tokenizer.train(
    files=['train_tokenizer.txt'],
    vocab_size=params['vocab_size'],
    min_frequency=params['min_frequency'],
    special_tokens=['[PAD]', '[SOS]', '[EOS]', '[UNK]'],
    wordpieces_prefix="##"
)

In [8]:
# 토크나이저 파라미터 확인

tokenizer._parameters

{'model': 'BertWordPiece',
 'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'cls_token': '[CLS]',
 'pad_token': '[PAD]',
 'mask_token': '[MASK]',
 'clean_text': True,
 'handle_chinese_chars': True,
 'strip_accents': None,
 'lowercase': True,
 'wordpieces_prefix': '##'}

In [9]:
# 패딩 토큰 인덱스 확인

print(tokenizer.token_to_id('[PAD]'))

0


In [10]:
# 스페셜 토큰 변수화

pad_idx = tokenizer.token_to_id('[PAD]')
sos_idx = tokenizer.token_to_id('[SOS]')
eos_idx = tokenizer.token_to_id('[EOS]')

In [11]:
# 토크나이저에 대해 패딩 옵션 설정
tokenizer.enable_padding(pad_id=pad_idx, pad_token='[PAD]')

In [12]:
# 'encode_batch' 함수 이용해 훈련 데이터셋에 대해 토크나이즈 수행

encoded_data = tokenizer.encode_batch(train_data)

In [13]:
# 데이터 개수 확인

print(f'훈련 데이터:\t{len(train_data)} 개')
print(f'훈련 라벨:\t{len(train_label)} 개')
print(f'인코딩 데이터:\t{len(encoded_data)} 개')

훈련 데이터:	484 개
훈련 라벨:	484 개
인코딩 데이터:	484 개


In [14]:
# 토크나이저 초기 훈련 결과 확인

print(f'토큰: {encoded_data[476].tokens}\n')
print(f'아이디: {encoded_data[476].ids}')

토큰: ['가게도', '예ᄈ', '##ᅳ', '##구', '분위기도', '넘', '좋았어요', '##ᆼ', '!', '!', '치킨', '가', '##ᆺ', '나오', '##ᆫ', '##거', '넘', '맛있구', '치즈', 'ᄑ', '##ᅳᆯ', '##레', '##이', '##트', '##도', '짱', '##맛', '크', '##래', '##커', '위', '##에', '어', '##ᆬ', '##어', '##진', '치즈', '제', '스타', '##ᆯ이', '##에요', 'ㅎㅎ', '이', '동네', '##에서', '요', '##론', '분위기', '맛집', '##을', '찾아', '##내', '##다니', '##🤗', '아', '그리고', '사장님', '진짜', '와', '##ᆼ', '친절해요', '##ᆷ', '!', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 

In [15]:
# 후처리 함수 정의

def postprocess(input_ids):
    input_ids = [sos_idx] + input_ids
    
    # 문장 최대 길이까지 슬라이싱
    input_ids = input_ids[:params['max_len']]

    # 패딩 토큰이 포함된 문장이라면 원 문장 말미에 <EOS> 토큰 삽입 
    if pad_idx in input_ids:
        pad_start = input_ids.index(pad_idx)
        input_ids[pad_start] = eos_idx

    # 패딩 토큰이 포함되지 않은 문장이라면, 시퀀스 말미에 <EOS> 토큰 삽입
    else:
        input_ids[-1] = eos_idx
    
    return input_ids

In [16]:
# 후처리 함수 이용해 토크나이즈 문장 후처리

processed_data = [postprocess(data.ids) for data in encoded_data]

In [17]:
# 후처리 결과 확인

print(f'후처리 결과: {processed_data[476]}\n')
print(f'후처리 결과 디코딩: {tokenizer.decode(processed_data[476])}')

후처리 결과: [1, 1798, 779, 163, 315, 778, 477, 547, 168, 4, 4, 881, 287, 184, 590, 2]

후처리 결과 디코딩: 가게도 예쁘구 분위기도 넘 좋았어용!! 치킨 갓 나오


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils import data

torch.manual_seed(32)
torch.cuda.manual_seed(32)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
# CNN 모델 정의

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.embedding = nn.Embedding(params['vocab_size'], params['embed_dim'], padding_idx=pad_idx)

        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1, 
                                              out_channels=params['num_filters'], 
                                              kernel_size=(fs, params['embed_dim'])) 
                                    for fs in params['filter_sizes']])
        
        self.fc = nn.Linear(len(params['filter_sizes']) * params['num_filters'], 1)
        
        self.dropout = nn.Dropout(params['dropout'])
        
    def forward(self, input_ids):
        # input_ids    = [배치 사이즈, 문장 길이]

        embedded = self.embedding(input_ids).unsqueeze(1)
        # embedded     = [배치 사이즈, 채널 개수, 임베딩 차원]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        # conved_n     = [배치 사이즈, 필터 개수, 문장 길이 - 필터 리스트[n] + 1]
        
        max_pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        # max_pooled_n = [배치 사이즈, 필터 개수]

        cat = self.dropout(torch.cat(max_pooled, dim = 1))
        # cat          = [배치 사이즈, 필터 개수 x len(필터 리스트)]

        return self.fc(cat)  # [배치 사이즈, 1]

In [20]:
# 문장 및 라벨 데이터 torch Tensor로 변환

processed_data = [torch.LongTensor(data).to(device) for data in processed_data]
train_label = [torch.FloatTensor([label]).to(device) for label in train_label]


# torch Tensor로 변환한 데이터 이용해 Iterator 정의

train_iter = data.DataLoader(processed_data, batch_size=params['batch_size'])
label_iter = data.DataLoader(train_label, batch_size=params['batch_size'])

In [21]:
model = CNN()
model.to(device)

criterion = nn.BCEWithLogitsLoss()
criterion.to(device)

optimizer = optim.Adam(model.parameters(), lr=params['lr'])

for epoch in range(params['num_epoch']):
    model.train()
    epoch_loss = 0
    
    for (batch, label) in zip(train_iter, label_iter):
        optimizer.zero_grad()

        logits = model(batch).squeeze(1)    # [배치 사이즈]
        labels = label.view(label.size(0))  # [배치 사이즈]

        loss = criterion(logits, labels)
        epoch_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    train_loss = epoch_loss / len(train_iter)        
    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f}')

Epoch: 01 | Train Loss: 0.525
Epoch: 02 | Train Loss: 0.316
Epoch: 03 | Train Loss: 0.218
Epoch: 04 | Train Loss: 0.136
Epoch: 05 | Train Loss: 0.091
Epoch: 06 | Train Loss: 0.055
Epoch: 07 | Train Loss: 0.036
Epoch: 08 | Train Loss: 0.023
Epoch: 09 | Train Loss: 0.019
Epoch: 10 | Train Loss: 0.014
Epoch: 11 | Train Loss: 0.009
Epoch: 12 | Train Loss: 0.009
Epoch: 13 | Train Loss: 0.008
Epoch: 14 | Train Loss: 0.007
Epoch: 15 | Train Loss: 0.005


In [22]:
# 결과 해석에 필요한 Captum API 임포트 및 정의

from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

token_reference = TokenReferenceBase(reference_token_idx=pad_idx)  # 레퍼런스 생성을 위한 모듈
lig = LayerIntegratedGradients(model, model.embedding)             # 결과 해석에 사용되는 IntegratedGradient 기법 모듈

In [23]:
vis_data_records_ig = []

label_vocab  = {0: '부정', 1: '긍정'}

def interpret_sentence(model, sentence, label = 0):       
    model.zero_grad()
    
    input_ids = tokenizer.encode(sentence)
    input_tokens = input_ids.tokens[:params['max_len']]
    
    input_ids = postprocess(input_ids.ids)
    input_indices_tensor = torch.LongTensor(input_ids).to(device).unsqueeze(0)

    # 단일 문장에 대한 예측 작업 수행
    pred = torch.sigmoid(model(input_indices_tensor)).item()
    pred_ind = round(pred)

    # 베이스 라인 역할을 할 Reference 생성: 주로 패딩 토큰으로 채워줌
    reference_indices = token_reference.generate_reference(params['max_len'], device=device).unsqueeze(0)

    # LayerIntegratedGradients 모듈 활용해 개별 단어의 속성값 및 델타값 근사치 계산
    attributions_ig, delta = lig.attribute(input_indices_tensor, reference_indices, n_steps=500, return_convergence_delta=True)

    print('pred: ', label_vocab[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))
    
    add_attributions_to_visualizer(attributions_ig, input_tokens, pred, pred_ind, label, delta, vis_data_records_ig)

    
    
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # 시각화 위해 샘플을 리스트에 추가
    vis_data_records.append(visualization.
                                VisualizationDataRecord(
                                    attributions,
                                    pred,
                                    label_vocab[pred_ind],
                                    label_vocab[label],
                                    label_vocab[1],
                                    attributions.sum(),       
                                    text,
                                delta
                                )
                           )

In [24]:
# 예제 문장 추가 및 분석 수행
torch.utils.backcompat.broadcast_warning.enabled=True
interpret_sentence(model, '맛있고 가게도 깔끔해서 또 방문할 것 같아요. 국물이 일품입니다..재방문의사있음', label=1)
interpret_sentence(model, '가게도 낡았고 진짜 별로.. 사장님도 불친절하고 기분만 나빴어요', label=0)
interpret_sentence(model, '와 진짜 인생맛집 간도 적당하고 분위기도 좋네요. 추천합니다 좋았습니다.ㅠ굿굿~~', label=1)
interpret_sentence(model, '불친절하고 소문에 비해 별로고.. 다른데 갈래요 비추합니다', label=0)
interpret_sentence(model, '사장님이 친절하시고 직원교육도 너무 잘시킨거 같아요 부모님모시고 왔는데 다음에 애인이랑 와야겠어요❤️', label=1)
interpret_sentence(model, '2인세트 있어서 편하게 먹었습니다 :) 맵기는 중간맛 선택했는데 많이 맵지 않았고 전체적으로 음식이 맛있더라구요 ❤️', label=1)
interpret_sentence(model, '한번쯤 경험할만한 그냥 단순한 지갑이 얇을때의 추억의 맛.. ', label=1)
interpret_sentence(model, '와~ 시그니처 자카파밀크..럼 향이 나는 커피인데, 그냥 마셔도 넘 맛있고, 우유 타도 맛있어요. 강추~~ 3층규모인데 사람이 많아 자리가 없었네요. 로스터리 직접 하고, 베이커리도 많고, 핸드드립바도 있구~ 분위기 좋아요. 근데 카페가 큰데 주차가 안되는 건 좀... ', label=1)


pred:  긍정 ( 0.88 ) , delta:  tensor([0.6237], dtype=torch.float64)
pred:  부정 ( 0.11 ) , delta:  tensor([1.2993], dtype=torch.float64)
pred:  긍정 ( 1.00 ) , delta:  tensor([0.2924], dtype=torch.float64)
pred:  부정 ( 0.18 ) , delta:  tensor([0.2980], dtype=torch.float64)
pred:  긍정 ( 1.00 ) , delta:  tensor([2.5802], dtype=torch.float64)
pred:  긍정 ( 0.99 ) , delta:  tensor([1.3523], dtype=torch.float64)
pred:  긍정 ( 0.61 ) , delta:  tensor([3.1320], dtype=torch.float64)
pred:  긍정 ( 1.00 ) , delta:  tensor([1.0285], dtype=torch.float64)


In [25]:
# 시각화 결과 표 변환
visualization.visualize_text(vis_data_records_ig)